In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver

import requests
import time
import re

import pandas as pd

In [2]:
url = 'http://www.nytimes.com/interactive/2016/01/28/upshot/donald-trump-twitter-insults.html?smid=tw-upshotnyt&smtyp=cur'

r = requests.get(url)


In [3]:
driver = webdriver.PhantomJS(executable_path='/usr/local/bin/phantomjs-2.1.1-macosx/bin/phantomjs')  # why the F did I put it there?
driver.get(url)
time.sleep(3)

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# save a local copy in case NYT takes the site down
with open('backup.html', 'w') as fp:
    fp.write(page_source)


In [4]:
entities = soup.find_all('div', class_='g-entity-item')

In [5]:
e = entities[1]
e.find('div', class_='g-entity-name')

<div class="g-entity-name" id="hillary-clinton">Hillary Clinton</div>

In [6]:
df = pd.DataFrame(columns=['id', 'name', 'text', 'links'])

for e in entities:
    element = e.find('div', class_='g-entity-name')

    e_name = element.contents
    e_id = element['id']
    
    insults = e.find('div', class_='g-insult-container').find_all('div', class_='g-insult-links-c')
    text = [x.a.contents for x in insults]
    links = [x.a['href'] for x in insults]
    
    # build a tmp data-frame
    tmp_df = pd.DataFrame(text, columns=['text'])
    tmp_df.loc[:, 'links'] = links
    tmp_df.loc[:, 'id'] = e_id
    tmp_df.loc[:, 'name'] = e_name    
    
    df = df.append(tmp_df, ignore_index=True)


In [7]:
len(df)

1787